In [ ]:
#import libraries
import os
import pandas as pd

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [ ]:
# Set hyperparameters
BATCH_SIZE = 4
NUM_TRAIN_EPOCHS = 1
LEARNING_RATE = 3e-5
WEIGHT_DECAY = 0.01
MAX_SOURCE_LENGTH = 512
MAX_TARGET_LENGTH = 64
TRAINING_DATASET_SIZE = 100
TESTING_DATASET_SIZE = 10


from datasets import load_dataset

dataset = load_dataset("d0r1h/ILC")

DATASET_PATH = pd.DataFrame(dataset['train'])
TEST_DATASET_PATH = pd.DataFrame(dataset['test'])


#CHANGE THIS BEFORE ANY HYPERPARAMETER CHANGE !!!!!!!!!!!!
OUTPUT_DIR_CHECKPOINT = "/content/cache"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/368 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/36.5M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2058 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1015 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd

# Save the training and testing datasets to CSV files
TRAIN_CSV_PATH = "/content/train_dataset.csv"
TEST_CSV_PATH = "/content/test_dataset.csv"

DATASET_PATH.to_csv(TRAIN_CSV_PATH, index=False)
TEST_DATASET_PATH.to_csv(TEST_CSV_PATH, index=False)


In [ ]:
from datasets import load_dataset

# Load the dataset from CSV files
legal_data = load_dataset("csv", data_files={"train": TRAIN_CSV_PATH, "test": TEST_CSV_PATH})

# Select the desired number of samples for training and testing
legal_data["train"] = legal_data["train"].select(range(TRAINING_DATASET_SIZE))
legal_data["test"] = legal_data["test"].select(range(TESTING_DATASET_SIZE))


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
legal_data["train"][0]

{'Title': 'Company may continue arbitration despite approval of CIRP : Supreme Court',
 'Summary': 'The issue whether a company involved in arbitration proceedings may continue the arbitration proceedings even after CIRP has been been approved by COC of the company was considered by the division bench of Supreme Court\xa0consisting of Justice AM Khanwilkar and Dinesh Maheswari in the matters between\xa0\xa0Fourth Dimension Solutions Ltd. v. Ricoh India Ltd.\xa0Civil Appeal No.\xa0–5908 of 2021 decided on 21.1.2022. The facts of this are defendant Ricoh owed the appellant Rs.511 crores. It is stated that NCLT has approved a resolution plan that dismisses the appellant’s (Fourth Dimension Solutions Ltd.) claims, even though the appellant is the highest operational creditor of the respondent company Ricoh India. The Supreme Court also upheld the resolution plan, despite the appellant’s objection that the same decision was pending before NCLAT. However, while the higher court approved the 

In [ ]:
import os

checkpoint_dir = "/content/cache"

if os.path.exists(checkpoint_dir):
    contents = os.listdir(checkpoint_dir)
    if contents:
        print("Contents of checkpoint directory:", contents)
    else:
        print("Checkpoint directory is empty.")
else:
    print("Checkpoint directory does not exist.")


Checkpoint directory is empty.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os

checkpoint_dir = OUTPUT_DIR_CHECKPOINT  # Assuming OUTPUT_DIR_CHECKPOINT is defined
checkpoint = "facebook/bart-large-cnn"

# Check if the checkpoint directory exists AND contains files
if os.path.exists(checkpoint_dir) and os.listdir(checkpoint_dir):
    try:
        model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_dir)
        print("Using checkpoint model from:", checkpoint_dir)
    except Exception as e:
        print(f"Error loading from checkpoint: {e}")  # Print any specific error message
        model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint) # Load from HF instead
        print("Falling back to pretrained model from Hugging Face:", checkpoint)

else:
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
    print("Using pretrained model from Hugging Face:", checkpoint)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Using pretrained model from Hugging Face: facebook/bart-large-cnn


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

preprocess


In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    # Prepare inputs by adding the prefix to each case
    inputs = [prefix + doc for doc in examples["Case"]]

    # Tokenize inputs with truncation and padding
    model_inputs = tokenizer(inputs, max_length=MAX_SOURCE_LENGTH, truncation=True, padding="max_length")

    # Tokenize labels (summaries) with truncation and padding
    labels = tokenizer(text_target=examples["Summary"], max_length=MAX_TARGET_LENGTH, truncation=True, padding="max_length")

    # Add labels to model inputs
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
tokenized_legal = legal_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
import evaluate
rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
from datasets import Dataset

# Convert IterableDataset to Dataset
train_dataset = Dataset.from_dict(tokenized_legal["train"].to_dict())
eval_dataset = Dataset.from_dict(tokenized_legal["test"].to_dict())


In [ ]:
import os
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers.trainer_callback import EarlyStoppingCallback

os.environ["WANDB_DISABLED"] = "true"

# Set up the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=checkpoint_dir,
    evaluation_strategy="steps",  # Change evaluation strategy to "steps"
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=1,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    predict_with_generate=True,
    fp16=False,
    logging_steps=1,
    load_best_model_at_end=True
)

# Set up the trainer with early stopping
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_legal["train"],
    eval_dataset=tokenized_legal["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer.train()
trainer.save_model(checkpoint_dir)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-22-274e275cf0be>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.695200,3.631601,0.237800,0.056900,0.138200,0.138800,111.900000
2,2.884100,3.560004,0.259000,0.069100,0.162700,0.161200,107.800000
3,3.571000,3.490189,0.256600,0.065700,0.170200,0.169100,96.400000
4,4.095400,3.439593,0.266600,0.065000,0.171700,0.170600,88.600000
5,3.660300,3.401178,0.236300,0.040700,0.148300,0.149800,89.100000
6,3.301700,3.379207,0.223100,0.038400,0.138600,0.140500,88.300000
7,3.514700,3.364189,0.246600,0.041100,0.153800,0.155000,95.400000
8,3.160100,3.352406,0.253700,0.036400,0.151500,0.152900,82.400000
9,3.739500,3.340262,0.227900,0.039700,0.148800,0.150300,87.400000
10,3.002800,3.327928,0.250200,0.046200,0.163100,0.164400,85.300000


In [ ]:
model_save_path = '/content/model'
trainer.save_model(model_save_path)

In [ ]:
train_metrics = trainer.evaluate(eval_dataset=tokenized_legal["train"])
print("Training ROUGE Scores:")
print("ROUGE-1:", train_metrics["eval_rouge1"])
print("ROUGE-2:", train_metrics["eval_rouge2"])
print("ROUGE-L:", train_metrics["eval_rougeL"])

test_metrics = trainer.evaluate(eval_dataset=tokenized_legal["test"])
print("Testing ROUGE Scores:")
print("ROUGE-1:", test_metrics["eval_rouge1"])
print("ROUGE-2:", test_metrics["eval_rouge2"])
print("ROUGE-L:", test_metrics["eval_rougeL"])

In [ ]:

import matplotlib.pyplot as plt

# ROUGE scores
labels = ["ROUGE-1", "ROUGE-2", "ROUGE-L"]
train_scores = [train_metrics["eval_rouge1"], train_metrics["eval_rouge2"], train_metrics["eval_rougeL"]]
test_scores = [test_metrics["eval_rouge1"], test_metrics["eval_rouge2"], test_metrics["eval_rougeL"]]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()

rects1 = ax.bar(x - width/2, train_scores, width, label='Training')
rects2 = ax.bar(x + width/2, test_scores, width, label='Testing')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('ROUGE Scores by Dataset')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

fig.tight_layout()


plt.show()


In [ ]:
text = "summarize: IN THE HIGH COURT OF JUDICATURE AT PATNA CRIMINAL MISCELLANEOUS No.377321 Arising Out of PS. Case No. 114 Year 2021 Thana BEGUSARAI MUFFASIL District 1. Munni Devi Female aged about 39 years wife of Mr. Subodh Singh Rinku Devi Female aged about 37 years wife of Mr. Santosh Singh 3. Komal Kumari Female aged about 18 years daughter of Mr. Santosh Chhoti Kumari Female aged about 14 years daughter of Mr. Subodh Singh All resident of Village Mohanpur P.S. Muffasil District Begusarai ... Petitioner s The State of Bihar ... Opposite Party s For the Petitioner s For the State For the Informant Mr. Ajay Kumar Thakur Advocate Mr. Md. Arif APP Mr. Nasrul Huda Khan Advocate CORAM: HONOURABLE MR. JUSTICE AHSANUDDIN AMANULLAH ORAL JUDGMENT Date : 06 08 2021 The matter has been heard via video conferencing. 2. The case has been taken up out of turn on the basis of motion slip filed by learned counsel for the petitioner on 26.07.2021 which was allowed 3. Heard Mr. Ajay Kumar Thakur learned counsel for the petitioners Mr. Md. Arif learned Additional Public Prosecutorfor the State and Mr. Nasrul Huda Khan learned counsel for the informant. 4. The petitioners apprehend arrest in connection with Muffasil PS Case No. 1121 dated 23.02.2021 instituted Patna High Court CR. MISC. No.377321 dt.06 08 2021 under Sections 147 148 149 341 323 324 448 379 302 427 354 of the Indian Penal Code 5. The allegation against the petitioners no. 1 and 2 is that they assaulted the mother of the informant with iron rod and stick and also snatched the gold chain and ear ring whereas the petitioner no. 3 is said to have assaulted the sister of the informant with stick and thereafter it is alleged that all the accused including other male members of the family of the petitioner had entered into the house and looted Rs. 27 000 and further against the petitioner no. 4 is that she had also bitten the sister of the informant and all the accused are said to have damaged the motorcycle belonging to the father of the informant by iron rod and stones and had also taken away three mobile phones from the house. It is alleged that the father of the informant was killed due to assault by the accused. 6. Learned counsel for the petitioners submitted that both the families were agnates and they have been falsely implicated. It was submitted that no reason has been assigned in the FIR for the occurrence whereas accused Santosh Singh of the present case has lodged Muffasil PS Case No. 1121 on the same day for the same occurrence where it is alleged that the informant of the present case along with his brothers had Patna High Court CR. MISC. No.377321 dt.06 08 2021 teased and harassed one of the petitioners and when the family members of the petitioners confronted the informant they were abused and assaulted along with other family members. Learned counsel submitted that the deceased is the own uncle of co accused Santosh Singh Subodh Singh and Randhir Kumar and both the sides are neighbours. Learned counsel submitted that the case is false also for the reason that there is no injury report of the mother of the informant and as far as the sister of the informant is concerned the injury report shows only one lacerated wound above left half upper lip 1â€ x 1 2â€ and no sign of bite was found by the doctor. Thus it was submitted that the injury itself appears to be superficial. It was submitted that the entire family including the petitioners who are female members having clean antecedent have been implicated with ulterior motive and as far as the death of the father of the informant is concerned the assault on him is attributed to the other male co accused and not the petitioners. It was submitted that another aspect of the matter is that in the FIR itself it is alleged that when other male members of the petitioners who are also co accused were assaulting the father of the informant the police had arrived and thus if the same is true then there could not have been any further assault on the father of the informant and Patna High Court CR. MISC. No.377321 dt.06 08 2021 the accused would also have been arrested there itself on 22.02.2021 but neither any case was lodged nor arrest made on 22.02.2021 and only after the case was instituted on 23.02.2021 the male members of the petitioners who are accused in the present case were arrested. It was submitted that the informant side had blocked the state highway and indulged in brick batting with the police for which Muffasil PS Case No. 1121 dated 23.02.2021 under Sections 147 148 341 323 504 353 332 of the Indian Penal Code was also instituted in which the informant of the present case and others are accused 7. Learned APP submitted that the petitioners are alleged to have assaulted the mother and bitten the sister of the 8. Learned counsel for the informant submitted that the father of the informant has died due to assault. However it was not controverted that the assault on the father of the informant is attributed to the male co accused in the case and not the petitioners and also that there is no injury report with regard to the mother of the informant as also that only a lacerated wound on the upper lip of the sister of the informant has been found by the doctor as noted in the injury report. 9. Having considered the facts and circumstances of Patna High Court CR. MISC. No.377321 dt.06 08 2021 the case and submissions of learned counsel for the parties in the tentative view of the Court the allegations against the petitioners who have been made accused along with their other male members are not corroborated by the material brought before the police inasmuch as there is no injury report with regard to the mother of the informant and further that the sister of the informant has suffered only a lacerated wound above the left half upper lip and the petitioners are ladies without any criminal antecedent the Court is inclined to allow their prayer for grant of pre arrest bail 10. Accordingly in the event of arrest or surrender before the Court below within six weeks from today the petitioners be released on bail upon furnishing bail bonds of Rs 25 000 each with two sureties of the like amount each to the satisfaction of the learned Chief Judicial Magistrate Begusarai in Muffasil PS Case No. 1121 subject to the conditions laid down in Section 438(2) of the Code of Criminal Procedure 1973 and furtherthat one of the bailors shall be a close relative of the petitioners that the petitioners and the bailors shall execute bond and give undertaking with regard to good behaviour of the petitioners andthat the petitioners shall cooperate with the Court and Patna High Court CR. MISC. No.377321 dt.06 08 2021 the police prosecution Any violation of the terms and conditions of the bonds or undertaking or failure to cooperate shall lead to cancellation of their bail bonds. 11. It shall also be open for the prosecution to bring any violation of the foregoing conditions of bail by the petitioners to the notice of the Court concerned which shall take immediate action on the same after giving opportunity of hearing to the petitioner(s 12. The petition stands disposed of in the (Ahsanuddin Amanullah J J. Alam "

In [ ]:
import torch

def summarize_text(input_text):

    text = "summarize: " + input_text

    inputs = tokenizer(text, max_length=MAX_SOURCE_LENGTH, truncation=True, return_tensors="pt").input_ids
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Move the model to the GPU
    model.to(device)

    # Move the inputs tensor to the GPU
    inputs = inputs.to(device)

    # Generate outputs
    outputs = model.generate(inputs, max_new_tokens=MAX_TARGET_LENGTH, do_sample=False)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return summary

In [ ]:
print(legal_data["train"][0]["Summary"])

In [ ]:
summarize_text(legal_data["train"][0]["Case"])

In [ ]:
summarize_text(legal_data["test"][0]["Summary"])

In [ ]:
summarize_text(legal_data["test"][0]["Case"])

In [ ]:
# Example Texts and Summaries
example_texts = [legal_data["test"][i]["Case"] for i in range(3)]
example_summaries = [summarize_text(legal_data["test"][i]["Case"]) for i in range(3)]

for i in range(3):
    print(f"Example {i + 1}:")
    print("Text:", example_texts[i])
    print("Summary:", example_summaries[i])
    print("="*50)

In [ ]:
# Create a DataFrame
df = pd.DataFrame({
    'Metric': labels,
    'Training': train_scores,
    'Testing': test_scores
})

# Save to CSV
df.to_csv('/content/rouge_scores.csv', index=False)

# Save ROUGE scores visualization
fig.savefig('/content/rouge_scores.png')

# Save text and summary examples to a text file
with open('/content/summary_examples.txt', 'w') as f:
    for i in range(3):
        f.write(f"Example {i + 1}:\n")
        f.write(f"Text: {example_texts[i]}\n")
        f.write(f"Summary: {example_summaries[i]}\n")
        f.write("="*50 + "\n")

In [ ]:
trainer.save_model('/content/drive/MyDrive/Bart')

In [ ]:
# Generate summaries for the entire training and testing datasets
train_summaries = [summarize_text(sample["Case"]) for sample in legal_data["train"]]
test_summaries = [summarize_text(sample["Case"]) for sample in legal_data["test"]]

# Create DataFrame for training and testing summaries
train_df = pd.DataFrame({
    "Original_Case": [sample["Case"] for sample in legal_data["train"]],
    "Original_Summary": [sample["Summary"] for sample in legal_data["train"]],
    "Generated_Summary": train_summaries
})

test_df = pd.DataFrame({
    "Original_Case": [sample["Case"] for sample in legal_data["test"]],
    "Original_Summary": [sample["Summary"] for sample in legal_data["test"]],
    "Generated_Summary": test_summaries
})

# Save DataFrames to CSV files
train_df.to_csv("/content/train_summaries.csv", index=False)
test_df.to_csv("/content/test_summaries.csv", index=False)
